# Search for range of float model

### Import Packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input # type: ignore
# Local imports
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.quantization_util import activation_range_search, gen_sample_paths

### Variable Paths, Names, Execution Environments

In [ ]:
# dict = path_definition(ds_rel_path='catsdogsconv/CatsDogs')
dict = path_definition()
BASE_PATH = dict['BASE_PATH']
PATH_DATASET = dict['PATH_DATASET']
PATH_TEST = dict['PATH_TEST']
PATH_RAWDATA = dict['PATH_RAWDATA']
PATH_JOINEDDATA = dict['PATH_JOINEDDATA']
PATH_SAVEDMODELS = dict['PATH_SAVEDMODELS']

### Load Float Model

In [ ]:
# Two first letters for dataset, next letter stands for parameter
# 3 numbers are for the number of epochs, and the last 4 digits are for validation loss
# e.g. CD4_P2_FT_003_val0.0336
model_name = ''
short_name = model_name[:-10]
parent_name = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_name}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)

### Activation Range for Random Input

In [ ]:
import numpy as np
input_shape = (1, 224, 224, 3)  # batch size 1 here
input_data = np.random.rand(*input_shape).astype(np.float32)

x = tf.convert_to_tensor(input_data)

print("Activation ranges per layer:")

# Manually feed-forward through layers
for i, layer in enumerate(model.layers):
    x = layer(x)  # call layer on tensor input

    # x is a tensor, get numpy array for min/max
    activations = x.numpy()

    print(f"Layer {i} ({layer.name}): min={activations.min()}, max={activations.max()}")

### Find Activation Range for Conv and Dense Layers & Store it

In [ ]:
sampled_files = gen_sample_paths()
activation_sw_range_dict = activation_range_search(sampled_files, model, model_name, mode='s', force=0)